In [1]:
import random
from pprint import pprint

from process_bigraph import Composite, ProcessTypes
from process_bigraph.emitter import emitter_from_wires, gather_emitter_results

from cdFBA import register_types
from cdFBA.utils import get_substrates, make_cdfba_composite

from spatial_transport.processes.diffusion import get_simple_diffusion_spec
from spatial_transport.processes.advection import get_simple_advection_spec
from spatial_transport import register_types as register_types2
from spatial_transport.utils import generate_voxels, generate_shared_environment, get_regular_edges

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [2]:
# # #Load E. coli model
# E_no_galE = cobra.io.load_model("iAF1260")
# # #Copy model
# E_no_LCTStex = E_no_galE.copy()
# # #Knock out galactose metabolism
# E_no_galE.genes.b0759.knock_out() # cannot metabolize galactose
# # #Knock out Lactose transport reaction LCTStex
# E_no_LCTStex.reactions.LCTStex.knock_out() # cannot uptake lactose
#
# #save models in SBML format
# write_sbml_model(E_no_galE, "E_no_galE.xml")
# write_sbml_model(E_no_LCTStex, "E_no_LCTStex.xml")

In [3]:
exchanges = ['EX_lcts_e', 'EX_gal_e']
all_species = ['EX_lcts_e', 'EX_gal_e', 'deltaGal', 'deltaLac']
substrates = get_substrates(model_file="iAF1260", exchanges=exchanges)

In [4]:
model_dict = {
    'deltaGal':'E_no_galE.xml',
    'deltaLac':'E_no_LCTStex.xml'
}

In [5]:
voxels = generate_voxels([20,1,0], spacing=1)
edges = get_regular_edges(voxels, periodic=False, spacing=1)

In [6]:
def generate_simple_cdfba_composite(voxels, model_dict, exchanges, volume):
    substrates = []
    species_list = [species for species in model_dict.keys()]
    for species, model in model_dict.items():
        substrates += get_substrates(model_file=model, exchanges=exchanges)
    base_spec = make_cdfba_composite(model_dict=model_dict, exchanges=exchanges, volume=volume, interval=0.1)
    for id in voxels:
        spec = base_spec.copy()
        shared_environment = generate_shared_environment(volume=1, substrates=substrates, species=species_list)
        spec["Shared Environment"] = shared_environment
        voxels[id].update(spec)
    return voxels

In [7]:
compartments = generate_simple_cdfba_composite(voxels, model_dict, exchanges, 1)

In [8]:
spec = {"Compartments": compartments, "Edges": edges}

In [9]:
for species in model_dict.keys():
    substrates.append(species)

In [10]:
substrates_dict = {
    "Lactose C12H22O11": 0.03,
    "D-Galactose": 0.06,
    "deltaGal": 0.001,
    "deltaLac": 0.001
}

In [11]:
spec["Simple Diffusion"] = get_simple_diffusion_spec(substrates=substrates_dict, interval = 0.1)

In [12]:
spec["Simple Advection"] = get_simple_advection_spec(spacing=1, substrates=substrates, advection=[0, 0.5, 0], boundary="default", interval=0.1)

In [13]:
spec["emitter"] = emitter_from_wires({
        "global_time": ["global_time"],
        'compartments': ["Compartments", "..", "Shared Environment"],
    })

In [14]:
core = ProcessTypes()
core = register_types(core)
core = register_types2(core)

In [15]:
sim = Composite(
        {
            "state": spec,
        },
        core=core
    )

In [16]:
sim.run(20)
results = gather_emitter_results(sim)[("emitter",)]